In [1]:
"""
A set partitioning model of a wedding seating problem

Authors: Stuart Mitchell 2009
"""

import pulp

max_tables = 5
max_table_size = 4
guests = 'A B C D E F G I J K L M N O P Q R'.split()

def happiness(table):
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(ord(table[0]) - ord(table[-1]))
                
#create list of all possible tables
possible_tables = [tuple(c) for c in pulp.allcombinations(guests, 
                                        max_table_size)]

#create a binary variable to state that a table setting is used
x = pulp.LpVariable.dicts('table', possible_tables, 
                            lowBound = 0,
                            upBound = 1,
                            cat = pulp.LpInteger)

seating_model = pulp.LpProblem("Wedding Seating Model", pulp.LpMinimize)

seating_model += sum([happiness(table) * x[table] for table in possible_tables])

#specify the maximum number of tables
seating_model += sum([x[table] for table in possible_tables]) <= max_tables, \
                            "Maximum_number_of_tables"

#A guest must seated at one and only one table
for guest in guests:
    seating_model += sum([x[table] for table in possible_tables
                                if guest in table]) == 1, "Must_seat_%s"%guest

seating_model.solve()

print("The choosen tables are out of a total of %s:"%len(possible_tables))
for table in possible_tables:
    if x[table].value() == 1.0:
        print(table)

/Users/joseph_cross/Anaconda/anaconda3/envs/optimus/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


The choosen tables are out of a total of 3213:
('M', 'N')
('E', 'F', 'G')
('A', 'B', 'C', 'D')
('I', 'J', 'K', 'L')
('O', 'P', 'Q', 'R')


# Test Weight Optimisation for Lorry Transport

In [3]:
model = pulp.LpProblem("Weight Dist Model", pulp.LpMaximize)

In [4]:
prod = ['A', 'B', 'C', 'D', 'E', 'F']

In [5]:
weight = {'A': 12800,'B': 10900, 'C':11400, 'D':2100, 'E':11300, 'F':2300}

In [7]:
profit = {'A': 77878,'B':82713, 'C':82728, 'D':68423, 'E':84119, 'F':77765}

In [8]:
x = pulp.LpVariable.dicts('ships_', prod, lowBound=0, cat = "Binary")

In [ ]:
# y = pulp.LpVariable.dicts('amount_')

In [9]:
model += pulp.lpSum([x[i] * profit[i] for i in prod])

In [10]:
model += pulp.lpSum([x[i]* weight[i] for i in prod]) <= 20000


In [28]:
model += x['E'] + x['D'] <= 1

In [36]:
model += x['D'] <= x['B']

In [37]:
model.solve()

1

In [38]:
for i in prod:
    print(f"{i} status {x[i].varValue}")

A status 0.0
B status 1.0
C status 0.0
D status 1.0
E status 0.0
F status 1.0


In [18]:
import numpy as np

In [39]:
x_np = np.array([x[i].varValue for i in x])

In [40]:
weights_np = np.array([weight[i] for i in weight])

In [41]:
profit_np = np.array([profit[i] for i in profit])

In [42]:
sum(x_np*weights_np)

15300.0

In [43]:
sum(x_np*profit_np)

228901.0

In [44]:
# Capacitated Plant Location

In [46]:
model = pulp.LpProblem("Cap Plant Model", pulp.LpMaximize)

In [47]:
plants = ['A', 'B', 'C', 'D']

In [49]:
cap = ['High', 'Low']

In [ ]:
cap_p = {'High':,'Low':} # actually df with location specific

In [48]:
x = pulp.LpVariable.dicts('loc_', [(i,j) for i in plants for j in plants], lowBound=0, cat = "Continuous")

In [50]:
y = pulp.LpVariable.dicts('cap_', [(i, s) for i in plants for s in cap] , cat = 'Binary')

In [ ]:
# Objective Function

In [ ]:
model += (pulp.lpSum([fixcost.loc[(i,s)]*y[(i,s)] for i in plants for s in cap])
         + pulp.lpSum([varcost.loc[(i,j)] *x[i,j] for i in plants for j in plants])
         )

In [51]:
# Constraint : total shipped to region j should equal demand of region j

In [ ]:
for j in plants:
    model += pulp.lpSum([x[(i,j) for i in plants]]) == demand.loc[j]

In [ ]:
# Constraint: capacity

In [ ]:
for i in plants:
    model +=  pulp.lpSum([x[(i,j)] for j in plants]) <= pulp.lpSum([y[(i, s)]*cap_p.loc[i,s] for s in cap])

# Whiskas example 

https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html

In [56]:
import pandas as pd

In [53]:
costs = {'chicken': 0.013, "beef": 0.008, "mutton": 0.010, 
        'rice': 0.002, 'wheat': 0.005, 'gel':0.001}

In [54]:
ingredients = ['chicken', 'beef', 'rice', 'wheat']
protein = [0.1,0.2,0,0.04]
fat = [0.08, 0.2, 0.01, 0.01]
fibre = [0.001, .005, .1, .15]
salt = [.002, .005, .002, .008]


In [78]:
df_ing = pd.DataFrame(index = ingredients, columns = ['protein', 'fat', 'fibre', 'salt'])

In [79]:
df_ing['protein'] = protein
df_ing['fat'] = fat
df_ing['fibre'] = fibre
df_ing['salt'] = salt

In [80]:
df_ing.loc['limit'] = [8,6,2,0.4]
df_ing.loc['direction'] = ['+', '+', '-', '-']

In [68]:
df_ing.loc['chicken', 'protein']

0.1

In [81]:
df_ing

,protein,fat,fibre,salt
chicken,0.1,0.08,0.001,0.002
beef,0.2,0.2,0.005,0.005
rice,0,0.01,0.1,0.002
wheat,0.04,0.01,0.15,0.008
limit,8,6,2,0.4
direction,+,+,-,-


In [106]:
model = pulp.LpProblem("Whiskas", pulp.LpMinimize)

In [107]:
# x1 = pulp.LpVariable('chicken', lowBound=0, upBound=100, cat ='Continuous')

In [108]:
# x2 = pulp.LpVariable('beef', lowBound=0, upBound=100, cat ='Continuous')

In [109]:
x_dict = pulp.LpVariable.dicts('_ing', ['chicken', 'beef'], cat='Continuous')

1.000𝑥1+1.000𝑥2=100.0  
0.100𝑥1+0.200𝑥2≥8.0 protein  
0.080𝑥1+0.100𝑥2≥6.0 fat  
0.001𝑥1+0.005𝑥2≤2.0 fibre  
0.002𝑥1+0.005𝑥2≤0.4 salt  

In [112]:
# model += x1*costs['chicken'] + x2*costs['beef']

In [113]:
model += pulp.lpSum([x_dict[i]*costs[i] for i in ['chicken', 'beef']])

/Users/joseph_cross/Anaconda/anaconda3/envs/optimus/lib/python3.7/site-packages/pulp/pulp.py:1380: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [114]:
model += pulp.lpSum([x_dict[i] for i in ['chicken', 'beef']]) == 100, "total_req"

In [115]:
df_ing.columns

Index(['protein', 'fat', 'fibre', 'salt'], dtype='object')

In [116]:
for ing in ['protein', 'fat', 'fibre', 'salt']:
    if df_ing.loc['direction', ing]=='+':
        model += pulp.lpSum([x_dict[i]*df_ing.loc[i, ing] for i in ['chicken', 'beef']]) >= df_ing.loc['limit', ing], f"{ing}_req"
    elif df_ing.loc['direction', ing]=='-':
        model += pulp.lpSum([x_dict[i]*df_ing.loc[i, ing] for i in ['chicken', 'beef']]) <= df_ing.loc['limit', ing], f"{ing}_req"

In [117]:
model.solve()

1

In [118]:
model.writeLP("WhiskasModel.lp")

[_ing_beef, _ing_chicken]

In [119]:
print("Status:", pulp.LpStatus[model.status])

Status: Optimal


In [120]:
# Each of the variables is printed with it's resolved optimum value
for v in model.variables():
    print(v.name, "=", v.varValue)

_ing_beef = 66.666667
_ing_chicken = 33.333333


In [121]:
print(model.objective.value())

0.966666665


In [127]:
for name,c in model.constraints.items():
    print(f"{name:>15}, {c.pi:>4.2f}, {c.slack:0.2f}")

      total_req, 0.02, -0.00
    protein_req, 0.00, -8.67
        fat_req, 0.00, -10.00
      fibre_req, 0.00, 1.63
       salt_req, -1.67, -0.00


## Full Whiskas model

In [153]:
model2 = pulp.LpProblem("Whiskas_Full", pulp.LpMinimize)

In [154]:
df_ing.index

Index(['chicken', 'beef', 'mutton', 'rice', 'wheat', 'gel', 'limit',
       'direction'],
      dtype='object')

In [155]:
costs = {'chicken': 0.013, "beef": 0.008, "mutton": 0.010, 
        'rice': 0.002, 'wheat': 0.005, 'gel':0.001}

ingredients = ['chicken', 'beef', 'mutton', 'rice', 'wheat', 'gel']
protein = [0.1,0.2,0.15,0,0.04,0]
fat = [0.08, 0.2, 0.11,0.01, 0.01, 0]
fibre = [0.001, .005, 0.003, .1, .15, 0]
salt = [.002, .005, 0.007,.002, .008, 0]


df_ing = pd.DataFrame(index = ingredients, columns = ['protein', 'fat', 'fibre', 'salt'])

df_ing['protein'] = protein
df_ing['fat'] = fat
df_ing['fibre'] = fibre
df_ing['salt'] = salt

df_ing.loc['limit'] = [8,6,2,0.4]
df_ing.loc['direction'] = ['+', '+', '-', '-']

In [156]:
x_dict = pulp.LpVariable.dicts('_ing', ingredients, lowBound=0, upBound=100,cat='Continuous')

In [157]:
model2 += pulp.lpSum([x_dict[i]*costs[i] for i in ingredients])

In [158]:
model2 += pulp.lpSum([x_dict[i] for i in ingredients]) == 100, "total_req"

In [159]:
for ing in ['protein', 'fat', 'fibre', 'salt']:
    if df_ing.loc['direction', ing]=='+':
        model2 += pulp.lpSum([x_dict[i]*df_ing.loc[i, ing] for i in ingredients]) >= df_ing.loc['limit', ing], f"{ing}_req"
    elif df_ing.loc['direction', ing]=='-':
        model2 += pulp.lpSum([x_dict[i]*df_ing.loc[i, ing] for i in ingredients]) <= df_ing.loc['limit', ing], f"{ing}_req"

In [160]:
model2.solve()

1

In [161]:
print("Status:", pulp.LpStatus[model2.status])

Status: Optimal


In [163]:
# help (model.solve)

In [164]:
# Each of the variables is printed with it's resolved optimum value
for v in model2.variables():
    print(v.name, "=", v.varValue)

_ing_beef = 40.0
_ing_chicken = 0.0
_ing_gel = 60.0
_ing_mutton = 0.0
_ing_rice = 0.0
_ing_wheat = 0.0


In [166]:
print(model2.objective.value())

0.38


In [167]:
for name,c in model.constraints.items():
    print(f"{name:>15}, {c.pi:>4.2f}, {c.slack:0.2f}")

      total_req, 0.02, -0.00
    protein_req, 0.00, -8.67
        fat_req, 0.00, -10.00
      fibre_req, 0.00, 1.63
       salt_req, -1.67, -0.00
